## Step 1. Loading and preparing data

Install all required packages for the project.

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Let's create dataframes for each region and look at the results for them

In [2]:
data1 = pd.read_csv('/datasets/geo_data_0.csv')
data2 = pd.read_csv('/datasets/geo_data_1.csv')
data3 = pd.read_csv('/datasets/geo_data_2.csv')
for item in [data1, data2, data3]:
        print(item.info())
        print(item.head())
        print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-nul

### Conclusion

Based on the data obtained, preprocessing is not required: we have three full-fledged datasets with 100,000 values each, without gaps, where signs and volume of oil are expressed as fractional numbers (that is, data coding is not required).

## Step 2. Train and validate the model

For each dataset (i.e. for each region), we will divide the data into **target attribute** - the volume of stocks in the *product* column - and into **features** - three unspecified factors. From the descriptive features, of course, we will omit the target one, as well as the unique id of the field, because it does not carry a training value for the model. The training and validation samples will be divided as 3:1, and the random_state parameter will be set here and below as 123.

In [3]:
### Region 1
target1 = data1['product']
features1 = data1.drop(['id', 'product'], axis=1)

features_train1, features_valid1, target_train1, target_valid1 = train_test_split(
    features1, target1, test_size=0.25, random_state=123)

In [4]:
### Region 2
target2 = data2['product']
features2 = data2.drop(['id', 'product'], axis=1)

features_train2, features_valid2, target_train2, target_valid2 = train_test_split(
    features2, target2, test_size=0.25, random_state=123)

In [5]:
### Region 3
target3 = data3['product']
features3 = data3.drop(['id', 'product'], axis=1)

features_train3, features_valid3, target_train3, target_valid3 = train_test_split(
    features3, target3, test_size=0.25, random_state=123)

Now, according to the condition, we will train linear regression models for each of the three regions, and also keep the predicted volume of reserves by wells

In [6]:
### Region 1
model1 = LinearRegression()
model1.fit(features_train1, target_train1) 
predictions1 = pd.Series(model1.predict(features_valid1), index=target_valid1.index)

In [7]:
### Region 2
model2 = LinearRegression()
model2.fit(features_train2, target_train2) 
predictions2 = pd.Series(model2.predict(features_valid2), index=target_valid2.index)

In [8]:
### Region 3
model3 = LinearRegression()
model3.fit(features_train3, target_train3) 
predictions3 = pd.Series(model3.predict(features_valid3), index=target_valid3.index)

We will receive and display the RMSE indicators for all three models.

In [9]:
result1 = math.sqrt(mean_squared_error(target_valid1, predictions1))
result2 = math.sqrt(mean_squared_error(target_valid2, predictions2))
result3 = math.sqrt(mean_squared_error(target_valid3, predictions3))

In [10]:
print('The average expected stock of raw materials in region 1 is {:.5} thousand barrels, while the RMSE is equal to {:.5}'.format(
predictions1.mean(), result1))
print('The average expected stock of raw materials in region 2 is {:.5} thousand barrels, while the RMSE is equal to {:.5}'.format(
predictions2.mean(), result2))
print('The average expected stock of raw materials in region 3 is {:.5} thousand barrels, while the RMSE is equal to {:.5}'.format(
predictions3.mean(), result3))

The average expected stock of raw materials in region 1 is 92.549 thousand barrels, while the RMSE is equal to 37.648
The average expected stock of raw materials in region 2 is 69.28 thousand barrels, while the RMSE is equal to 0.89541
The average expected stock of raw materials in region 3 is 95.099 thousand barrels, while the RMSE is equal to 40.128


### Conclusion

So, based on training models for each of the three regions, we can conclude that regions 1 and 3, on average, have larger oil deposits than region 2. However, the model for region 2 has a very close to zero error rate, which means , the model itself is more accurate. To make a choice in favor of a particular region, it is worth analyzing the data more deeply.

## Step 3. Preparation for profit calculation

Taking into account the budget and income from one thousand barrels, we will calculate the minimum amount of reserves for the region to interest us. To do this, we will divide the budget by the profitability of one thousand barrels, and then by **another 200** - by the number of fields in the region that will be developed for oil production.

In [12]:
budget = 10000000000
unit_profit = 450000
print('Break-even well development requires at least {} thousand barrels of oil'.format(
budget/unit_profit/200))

Break-even well development requires at least 111.11111111111111 thousand barrels of oil


So, a little over 111,000 barrels of oil is needed to start development and at least not go negative. Given that each region has deposits on average between 60,000 and 90,000 barrels each, it seems that the project could be profitable, but only if the largest wells are explored. To explore the possibility of this in more detail, it is worth calculating the immediate return by multiplying the average expected reserves by the profitability of one thousand barrels.

In [13]:
print('Exploration of reserves in region 1 will bring an average of {} rubles from each well'.format(predictions1.mean()*unit_profit))
print('Exploration of reserves in region 1 will bring an average of {} rubles from each well'.format(predictions2.mean()*unit_profit))
print('Exploration of reserves in region 1 will bring an average of {} rubles from each well'.format(predictions3.mean()*unit_profit))

Exploration of reserves in region 1 will bring an average of 41647212.851023376 rubles from each well
Exploration of reserves in region 1 will bring an average of 31176008.37294289 rubles from each well
Exploration of reserves in region 1 will bring an average of 42794369.701161176 rubles from each well


### Conclusion

The profitability of each well will be from 31 to 42 million rubles, based on average values. As for oil reserves, on average they are below the break-even point - for region 2, almost twice - so further analysis is needed to calculate the maximum profitable wells.

## Step 4. Calculation of profit and risks

First, let's write a function that will calculate the total profit from the largest fields in the region. The logic here is to sort the 200 largest (predicted) fields, then add up the **real** volumes of oil in them and multiply by the profitability per thousand barrels (for convenience, divide by 1 billion to make it easier to compare with the budget), and then **subtract** those same 10 billion - in order to get not revenue, but profit.

In [14]:
def profit(predictions, target):
    max_predictions = pd.Series(predictions).sort_values(ascending=False)[:200]
    max_volumes = target[max_predictions.index]
    total_oil = sum(max_volumes)
    total_profit = total_oil * unit_profit - budget
    return total_profit/1000000000

Let's apply the function to each region:

In [15]:
print('Profit expected from the largest wells in region 1 is equal to {} billion rubles'.format(
profit(predictions1, target1)))
print('Profit expected from the largest wells in region 2 is equal to {} billion rubles'.format(
profit(predictions2, target2)))
print('Profit expected from the largest wells in region 3 is equal to {} billion rubles'.format(
profit(predictions3, target3)))

Profit expected from the largest wells in region 1 is equal to 3.534670917261379 billion rubles
Profit expected from the largest wells in region 2 is equal to 2.415086696681551 billion rubles
Profit expected from the largest wells in region 3 is equal to 2.370343863021372 billion rubles


So, all three regions are potentially profitable, and region 1 is leading by a wide margin - at best, it promises 3.5 billion rubles of profit.  
Now it is also necessary to assess the risks. To do this, we will carry out the bootstrap procedure - we will select from 500 deposits in each region in a thousand iterations, and examine their profitability.

In [16]:
### So that all 1000 iterations do not give exactly the same sample, random state must be set
### not as a fixed value, but as a function from the numpy module
state = np.random.RandomState(123)

### Region 1
incomes1 = []
for i in range(1000):
    predictions_subsample1 = pd.Series(predictions1).sample(500, random_state=state, replace=True)
    incomes1.append(profit(predictions_subsample1, target1))

### Region 2
incomes2 = []
for i in range(1000):
    predictions_subsample2 = pd.Series(predictions2).sample(500, random_state=state, replace=True)
    incomes2.append(profit(predictions_subsample2, target2))

### Region 3    
incomes3 = []
for i in range(1000):
    predictions_subsample3 = pd.Series(predictions3).sample(500, random_state=state, replace=True)
    incomes3.append(profit(predictions_subsample3, target3))
    
print('According to the bootstrap results:')
print('Average profit in region 1 will be {:.5} billion rubles'.format(pd.Series(incomes1).mean()))
print('Average profit in region 2 will be {:.5} billion rubles'.format(pd.Series(incomes2).mean()))
print('Average profit in region 3 will be {:.5} billion rubles'.format(pd.Series(incomes3).mean()))

According to the bootstrap results:
Average profit in region 1 will be 0.47742 billion rubles
Average profit in region 2 will be 0.46748 billion rubles
Average profit in region 3 will be 0.36213 billion rubles


It follows from the previous step that, according to the average values within the bootstrap, all three regions will bring profit, but rather insignificant relative to the budget (only within 3-5% with huge investments). Regions 1 and 2 are the most profitable, almost without separation from each other, and region 3 is the least profitable (360 million rubles in total).

Now we should look at the 95% confidence interval as well, which means calculating the 2.5% and 97.5% quantiles.

In [17]:
lower1 = pd.Series(incomes1).quantile(0.025)
upper1 = pd.Series(incomes1).quantile(0.975)

lower2 = pd.Series(incomes2).quantile(0.025)
upper2 = pd.Series(incomes2).quantile(0.975)

lower3 = pd.Series(incomes3).quantile(0.025)
upper3 = pd.Series(incomes3).quantile(0.975)

print('The 95% confidence interval for region 1 was from {:.5} to {:.5} billion rubles'.format(lower1, upper1))
print('The 95% confidence interval for region 2 was from {:.5} to {:.5} billion rubles'.format(lower2, upper2))
print('The 95% confidence interval for region 3 was from {:.5} to {:.5} billion rubles'.format(lower3, upper3))

The 95% confidence interval for region 1 was from -0.057994 to 0.97482 billion rubles
The 95% confidence interval for region 2 was from 0.068764 to 0.87149 billion rubles
The 95% confidence interval for region 3 was from -0.18299 to 0.87097 billion rubles


According to the confidence interval, the data are stable, but disappointing: only the profitability of the second region passes the breakeven point, in the other two cases, negative values also fall into the confidence interval.
<br/><br/>
Finally, let's take a look at the riskiness of developing regions - we will write a function where the total profitability for all fields in each of the samples is viewed in a cycle, the number of unprofitable options (less than 10 billion) is calculated, and their share is compared with a threshold of 2.5%.

In [18]:
def risk(profits):
    minus_count = 0
    for item in profits:
        if item < 0:
            minus_count += 1
    risk = minus_count/1000
    if risk < 0.025:
        print('The region is worth developing, the probability of losses is', risk)
    else:
        print('The region should not be developed, the probability of losses is more than 2.5%')  

In [19]:
print('Region 1:')
risk(incomes1)
print('')
print('Region 2:')
risk(incomes2)
print('')
print('Region 3:')
risk(incomes3)

Region 1:
The region should not be developed, the probability of losses is more than 2.5%

Region 2:
The region is worth developing, the probability of losses is 0.01

Region 3:
The region should not be developed, the probability of losses is more than 2.5%


### Conclusion

The bootstrap procedure with 1000 repetitions showed the following results.  
The maximum revenue from each region can be very, very attractive - about 13.5 billion rubles, or 3.5 billion rubles of profit (region 1 is the most profitable). At the same time, all three regions are not very resistant to risks: two of them are unprofitable, even in the 95% confidence interval. If we talk directly about the probability of losses, then only region 2 does not cross the threshold of 2.5% of unprofitable deposits (which, in principle, followed from the confidence interval), the other two regions are potentially unprofitable.

## General conclusion

During the project for a mining company, data on oil samples were examined in three regions: in each of 10,000 fields, where the quality of oil and the volume of its reserves were measured. The aim of the project was to determine the most favorable region for exploration of reserves. To do this, a linear regression model was built for each region and the RMSE value was measured. Despite the fact that the average values of reserves in regions 1 and 3 were almost one and a half times higher than in region 2, their standard error was almost equal to this very difference. This means that the model for region 2 turned out to be the most accurate.
<br/><br/>
As for the profitability of oil production in the regions, on average, one well promises to bring from 30 (region 2) to 40 (regions 1, 3) million rubles. At the same time, the maximum expected profit in region 1 will be more than 3 billion rubles, and about 2.4 billion rubles in regions 2 and 3.
<br/><br/>
Finally, using the bootstrap, data on the riskiness of activities were obtained. Two regions promise to be unprofitable (1 and 3), because the number of unprofitable options for selecting deposits is more than the established threshold of 2.5%. The same is evidenced by the confidence interval of 95% (however, the upper quantile of the interval is not very promising and is far from the maximum possible profit).
<br/><br/>
To sum up, development can be carried out, in essence, only in ***region 2***. Firstly, its model was much stronger and more accurate than the others, with a very small RMSE. Secondly, its profitability is quite high, even if it is inferior to the region by one - quite a bit in terms of average expected profit and more than 1 billion in terms of the maximum possible one. Further, already after the bootstrap, which, if possible, excluded random elements, it became clear that region 2 is the only one, the probability of which is less than 2.5%, and there are no negative values in the 95% confidence interval. Accordingly, guided by these considerations, I would opt for **region 2**, which always shows the only non-negative results for all negative indicators.